In [ ]:
%reset -f

In [ ]:
def is_main_module():
    """
    Returns whether this notebook is the main module
    i.e. not being run from anotherr notebook
    """
    return __name__ == '__main__' and '__file__' not in globals()

SAVE_FIGURES = False
SAVE_FIGURE_DPI = 300
DEBUG_PRINT_PLOTS = True
SAVE_FIGURE_FORMAT = "PDF"  # PDF, SVG, PNG, for saved images

In [ ]:
# Import libraries
import sys
# Add the path to the Python module directory
sys.path.insert(0, '../Python')

import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

#import libraries
import sys
# add the path to the Python module directory
sys.path.insert(0, '../Python')

import matplotlib.pyplot as plt
import numpy as np
import math

import Constants
import Utils

import SphericalEarthModel as SEM
import Prf as PRF
import Nesz as NESZ
import Aasr as AASR
import Rasr as RASR

import planararrayantenna as PAA
import rectangularantenna as RA
import lineararrayantenna as LAA

def green(val):
    color = 'lightgreen'
    return 'background-color: %s' % color

def yellow(val):
    color = 'yellow'
    return 'background-color: %s' % color

def red(val):
    color = 'red'
    return 'background-color: %s' % color

def blue(val):
    color = 'lightblue'
    return 'background-color: %s' % color

## Altitude vs Incidence Angle

In [ ]:
if is_main_module():
    incidenceAngles = Utils.deg2rad(np.arange(0, 100, 1))             # angle from 0 to 100 degrees
    altitudes       = np.arange(500e3, 4500e3, 500e3, dtype='int64')  # altitudes from 500 to 4000 km

    # calculate the look angles for the given incidence angles
    lookAngles = []
    for alt in (altitudes):
        lookAngles.append(SEM.LookAngle(alt, incidenceAngles))
        
    # plot the look angles vs incidence angles
    fig, ax = plt.subplots(figsize=(8,4))
    for i in range(altitudes.shape[0]):
        ax.plot(Utils.rad2deg(incidenceAngles), 
                Utils.rad2deg(lookAngles[i]), 
                label='Altitude = %d km' % (altitudes[i] / 1000))
        
    # add incidence angle limit lines
    ax.axvline(x=20, linestyle="dashed", color="blue")
    ax.axvline(x=49, linestyle="dashed", color="blue")
    
    # grid, labels and legend
    plt.legend(framealpha=1.)
    plt.xlabel(r'Incidence Angle, $\eta$ (deg)')
    plt.ylabel(r'Look Angle, $\gamma$ (deg)')
    ax.grid(linestyle='dotted')
    plt.tight_layout()
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_19_ia_vs_la.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_19_ia_vs_la.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_19_ia_vs_la.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

## Initial System Parameters

In [ ]:
if is_main_module():
    Altitude        = 580e3                                 # 580 kilometer
    IncidenceAngles = Utils.deg2rad(np.arange(10, 91, 1))   # 10-90 degrees
    ReqGroundSwath  = 70e3                                  # 70 kilometer
    TxDutyCycle     = 0.05                                  # 5%
    Frequency       = 5.4e9                                 # C-band
    Wavelength      = Constants.SPEED_OF_LIGHT / Frequency  # meters
    TxPowerAvg      = 500                                   # Watts
    TxPowerPeak     = TxPowerAvg / TxDutyCycle
    NoiseFigure     = 4.3                                   # dB
    Losses          = 4.1                                   # dB

## PRF vs Incidence Angle Timing Diagram

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to cover the entire figure area
    numPrfPulses   = 41
    numNadirEchoes = 25
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngles = SEM.LookAngle(Altitude, IncidenceAngles)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(ReqGroundSwath,
                                              Altitude, 
                                              IncidenceAngles,
                                              lookAngles)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRangesDutyCycle(numPrfPulses,
                                                       nearDelay,
                                                       farDelay,
                                                       TxDutyCycle)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRangesDutyCycle(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       TxDutyCycle,
                                       nadirDelay)


    # get the maximum PRF
    prfMax = PRF.PrfMaxDutyCycle(TxDutyCycle, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay: {:.6f} s".format(nadirDelay))
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (8,4))    
    
    # secondary axis for ground range values
    ax2 = ax1.twiny()
    # Add some extra space for the second axis at the bottom
    fig.subplots_adjust(bottom=0.2)

    # locations on the first axis, where the second axis values are calculated
    new_tick_locations = np.array([15, 20., 25., 30., 32.45, 35., 37, 40., 45., 50.])

    # convert between the one axis to the next
    def tick_function(altitude, x):
        # convert look angle to incidence angle
        IA = SEM.IncidenceAngle(altitude, Utils.deg2rad(x))
        V = SEM.IncidenceAngleToGroundRange(altitude, IA)
        return ["%.0f" % z for z in V / 1000]

    # Move twinned axis ticks and label from top to bottom
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")

    # Offset the twin axis below the host
    ax2.spines["bottom"].set_position(("axes", -0.2))
    # sett the ticks values and locations on the second axis
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
    ax2.set_xlabel("Ground Range (km)")
    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(Utils.rad2deg(lookAngles), prfLower, color="black", alpha=0.6)
        ax1.plot(Utils.rad2deg(lookAngles), prfUpper, color="black", alpha=0.6)
        plt.fill_between(Utils.rad2deg(lookAngles), prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(Utils.rad2deg(lookAngles), nadirLower, color="red", alpha=0.6)
        ax1.plot(Utils.rad2deg(lookAngles), nadirUpper, color="red", alpha=0.6)
        plt.fill_between(Utils.rad2deg(lookAngles), nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    

    # plot PRFM Maximum
    ax1.plot(Utils.rad2deg(lookAngles), prfMax, color='tab:green', linestyle='dashed')     
    
    # plot the look angle limitations
    laMin = 32.5
    laMax = 37
    prf = 1550
    ax1.axhline(y=prf, color="black", linestyle="dashed")
    ax1.axvline(x=laMin, color="black", linestyle="dashed")
    ax1.axvline(x=laMax, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Incidence Angle')
    ax1.set_xlabel(r'Look Angle, $\theta_{i,m}$ (deg)')
    ax1.set_ylabel(r'PRF (Hz)')    
    plt.grid(linestyle='dotted')
    
    # limit the figure area
    ax1.set_ylim(500, 3000)  
    ax1.set_xlim(18, 51)
    
    plt.tight_layout()
    plt.tight_layout()
    ax1.margins(0)
    ax2.set_xlim(ax1.get_xlim())
    ax2.grid(False)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_20_com_prf_vs_loo.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_20_com_prf_vs_loo.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_20_com_prf_vs_loo.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)
    else:
        plt.show()

In [ ]:
# ground swath estimation
lowLa = np.deg2rad(laMin)   # lower bound look angle in radians
highLa = np.deg2rad(laMax)  # upper bound look angle in radians

lowIa = SEM.IncidenceAngle(Altitude, lowLa)   # lower bound incidence angle in radians
highIa = SEM.IncidenceAngle(Altitude, highLa) # upper bound incidence angle in randians

lowSR = SEM.SlantRange(Altitude, lowIa, lowLa)          # lower bound slant range in meters
highSR = SEM.SlantRange(Altitude, highIa, highLa)       # upper bound slant range in meters
print("Swath: {:.3f} km".format((highSR- lowSR)/1000))

lowGR = SEM.IncidenceAngleToGroundRange(Altitude, lowIa)   # lower bound ground range in meters
highGR = SEM.IncidenceAngleToGroundRange(Altitude, highIa) # upper bound ground range in meters
swath = highGR - lowGR
print("Ground Swath: {:.3f} km".format(swath/1000))

In [ ]:
print(Wavelength)

In [ ]:
# Get the ground swath width for the different look angles and TX elevation beamwidths
if is_main_module():
    Test_el_bw_range = np.arange(1, 61, 1)                 # beamwidth ranges between 1 and 60 degrees
    Test_look_angles = np.arange(20, 40, 5)                # elevation look angle ranges between 20 and 35 degrees
    AntennaOffset    = (laMin + laMax)/2                   # antenna mounting angle, middle of look angle range
    Test_look_angles = np.append(Test_look_angles, AntennaOffset)
    Test_altitudes   = np.asarray([Altitude])              # test altitudes. Min, mid and Max altitudes
    
    Test_altitude_swaths = []
    for alt in Test_altitudes:
        # for every altitude calculate the different swath over the look angle ranges, for the different beam widths
        Test_ground_swaths = [] 
        for la in Test_look_angles:
            _, _ground_swath = SEM.SwathGroundSwathFromBeamwidth(alt, 
                                                                 Utils.deg2rad(la), 
                                                                 Utils.deg2rad(Test_el_bw_range))
            Test_ground_swaths.append(_ground_swath)
        # append to the swath list
        Test_altitude_swaths.append(Test_ground_swaths)
    # convert the swath list to an array
    Test_altitude_swaths = np.asarray(Test_altitude_swaths)

    # Plot the ground swath widths vs beamwidths & look angles
    fig, ax1 = plt.subplots(1,1,figsize = (8,3))
    for i in range(Test_altitude_swaths.shape[0]):
        for j in range(Test_altitude_swaths[i].shape[0]):
            ax1.plot(Test_el_bw_range, 
                     Test_altitude_swaths[i][j,:] / 1000, 
                     label=r"Look Angle = %.2f deg" % (Test_look_angles[j]))
            
    # add the beamwidth and ground-swath intersection
    ax1.axvline(x=(laMax - laMin), linestyle="dotted")
    ax1.axhline(y=swath/1000, linestyle="dotted")
    
    # legends, labels and grids
    ax1.legend(loc=2)
    ax1.set_xlabel(r"Elevation Beamwidth (deg)")
    ax1.set_ylabel(r"Ground swath Width (km)")
    ax1.grid(linestyle='dotted')
    
    # limit the figure area
    ax1.set_ylim(0, 200)
    ax1.set_xlim(1,10)
    
    plt.tight_layout()
    
    # calculate the swath and ground-swath widths from the look ange and the beamwidth
    ReqTxBeamwidth_nom = (laMax - laMin)
    sw, gw =  SEM.SwathGroundSwathFromBeamwidth(Test_altitudes, 
                                            Utils.deg2rad(AntennaOffset), 
                                            Utils.deg2rad(ReqTxBeamwidth_nom))
    print("Swath = {:.2f} km and Ground swath = {:.2f} km @ Look angle = {} deg and Elevation BW = {} deg"
          .format(sw[0]/1000, gw[0]/1000, AntennaOffset, ReqTxBeamwidth_nom))

## Derived System Parameters and System Requirements

In [ ]:
if is_main_module():
    # system requirements
    ReqNesz         = -20       # Required NESZ in dB
    ReqRasr         = -20       # Required RASR in dB
    ReqAasr         = -20       # Required AASR in dB
    ReqResolutionAz = 10        # Required Azimuth resolution in meters   (cross-range) 
    ReqResolutionRg = 10        # Required Elevation resolution in meters (ground range)
    ReqGroundSwath  = swath     # Required Ground swath width in meters
    
    # Derived parameters
    Prf                    = prf
    MidSwathIncidenceAngle = SEM.IncidenceAngle(Altitude, Utils.deg2rad(AntennaOffset))
    ChirpBandwidth         = Constants.SPEED_OF_LIGHT / (2*ReqResolutionRg*np.sin(MidSwathIncidenceAngle))    
    TxPowerAvg             = TxPowerPeak * TxDutyCycle
    PulseLen               = TxDutyCycle / Prf
    CompressionGain        = (PulseLen * ChirpBandwidth)

    #------------------------------------------------------------
    # geometry code
    IncidenceAngleRange = np.arange(20, 49.1, 0.1)             # Scene incidence angle range in degrees
    IncidenceAngleRangeRad = IncidenceAngleRange * (np.pi/180)

    # get the look angles from the incidence angles
    LookAngleRangeRad = SEM.LookAngle(Altitude, IncidenceAngleRangeRad)
    LookAngleRange = LookAngleRangeRad * (180/np.pi)
    
    # the mid-swath look angle is equivalent to the antenna mounting angle
    MidSwathLookAngle = Utils.deg2rad(AntennaOffset)

## Phased Array Design

### Panel Concept

A single panel consists of 7 elements in elevation and 4 elements in azimuth. Beamsteering performed only in elevation. The elements are spaced 23 mm apart, this is less than $\lambda/2$ apart, so no grating lobes visible. This gives the antenna dimensions as:

Length (az) = 92 mm, Height (el) = 161 mm

In [ ]:
# antenna panel code
panel = lambda:None
panel.Length = 0.092
panel.Height = 0.161
panel.NumElElements = 7
panel.NumAzElements = 4
panel.ElElementSpacing = panel.Height / panel.NumElElements
panel.AzElementSpacing = panel.Length / panel.NumAzElements

### Antenna Face

The antenna face consists of 48 panels in azimuth and 5 panels in elevation, making the overall antenna dimension:

Length (az) = 4.416 m, Height (el) = 0.805 m

In [ ]:
# antenna face code
NumAzPanels = 48
NumElPanels = 5

### RX Antenna Model

Makes use of the entire antenna face.

In [ ]:
# RX Antenna code
efficiency = 0.64
ReqRxNumElems = panel.NumElElements * NumElPanels

# initialise the RX antenna object
aesaRxAnt = PAA.PlanarArrayAntenna(
                num_el_elems    = ReqRxNumElems,                                    
                num_az_elems    = panel.NumAzElements * NumAzPanels,      
                el_elem_spacing = panel.ElElementSpacing, 
                az_elem_spacing = panel.AzElementSpacing,                                                                 
                efficiency      = efficiency)

# swath covered by RX beam
_, rxGroundSwath = SEM.SwathGroundSwathFromBeamwidth(
                        Altitude, Utils.deg2rad(AntennaOffset) , 
                        aesaRxAnt.elevation_beamwidth(Wavelength))

In [ ]:
# display the RX antenna characteristics in a table format
df = pd.DataFrame (
    {
        "Parameter" : ['Length (Azimuth)', 
                       'Height (Elevation)', 
                       'Elevation Element Spacing', 
                       'Azimuth Element Spacing',
                       '# RX Elevation Elements', 
                       'RX Elevation Beamwidth', 
                       '# RX Azimuth Elements', 
                       'RX Azimuth Beamwidth',                       
                       'RX Gain', 
                       'Ground Swath'],
        "Value" : [aesaRxAnt.length(), 
                   aesaRxAnt.height(), 
                   panel.ElElementSpacing, 
                   panel.AzElementSpacing, 
                   ReqRxNumElems, 
                   Utils.rad2deg(aesaRxAnt.elevation_beamwidth(Wavelength)),
                   panel.NumAzElements * NumAzPanels, 
                   Utils.rad2deg(aesaRxAnt.azimuth_beamwidth(Wavelength)),
                   10*np.log10(aesaRxAnt.gain(Wavelength)), 
                   rxGroundSwath / 1000],
        "Unit" : ['m', 'm', 
                  'm', 'm',
                  '','deg', 
                  '','deg', 
                  'dBi', 'km']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles) \
  .style.applymap(yellow, subset=pd.IndexSlice[0:1, :]) \
        .applymap(blue, subset=pd.IndexSlice[4:7, :]) \
        .applymap(green, subset=pd.IndexSlice[8:9, :])

### TX Antenna Model

Elevation beamwidth determined by the swath requirements

In [ ]:
# TX Antenna code
efficiency = 0.64

ReqTxBeamwidth_70km = ReqTxBeamwidth_nom

# get the number of elements for the beamwidth to cover 70 km swath
ReqTxNumElems_70km = PAA.PlanarArrayAntenna.num_elements_for_beamwidth(
                        Wavelength, 
                        Utils.deg2rad(ReqTxBeamwidth_70km),
                        panel.ElElementSpacing, efficiency)

ReqTxNumPanels_70km = np.ceil(ReqTxNumElems_70km / panel.NumElElements)
print("Num elements required: {}".format(ReqTxNumElems_70km))
print("Num panels required: {}".format(ReqTxNumPanels_70km))

TxNumElPanels = int(ReqTxNumPanels_70km)

# initialise the TX antenna object
aesaTxAnt = PAA.PlanarArrayAntenna(
                num_el_elems    = ReqTxNumElems_70km,
                num_az_elems    = panel.NumAzElements * NumAzPanels,                                    
                el_elem_spacing = panel.ElElementSpacing, 
                az_elem_spacing = panel.AzElementSpacing,
                efficiency      = efficiency)

# swath covered by RX beam
_, txGroundSwath = SEM.SwathGroundSwathFromBeamwidth(
                        Altitude, Utils.deg2rad(AntennaOffset),
                        aesaTxAnt.elevation_beamwidth(Wavelength))

In [ ]:
# display the RX antenna characteristics in a table format
df = pd.DataFrame (
    {
        "Parameter" : ['Length (Azimuth)', 
                       'Height (Elevation)', 
                       'Elevation Element Spacing', 
                       'Azimuth Element Spacing',
                       '# TX Elevation Elements', 
                       'TX Elevation Beamwidth',
                       '# TX Azimuth Elements',  
                       'TX Azimuth Beamwidth',
                       'TX Gain', 
                       'Ground Swath'],
        "Value" : [aesaTxAnt.length(), 
                   aesaTxAnt.height(),
                   panel.ElElementSpacing, 
                   panel.AzElementSpacing, 
                   ReqTxNumElems_70km,
                   Utils.rad2deg(aesaTxAnt.elevation_beamwidth(Wavelength)),
                   panel.NumAzElements * NumAzPanels, 
                   Utils.rad2deg(aesaTxAnt.azimuth_beamwidth(Wavelength)),
                   10*np.log10(aesaTxAnt.gain(Wavelength)), 
                   txGroundSwath / 1000],
        "Unit" : ['m', 'm', 
                  'm', 'm',
                  '','deg', 
                  '','deg', 
                  'dBi', 'km']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles) \
  .style.applymap(yellow, subset=pd.IndexSlice[0:1, :]) \
        .applymap(blue, subset=pd.IndexSlice[4:7, :]) \
        .applymap(green, subset=pd.IndexSlice[8:9, :])

### NESZ

In [ ]:
# Calculate the antenna patterns used for calculating the NESZ
NumSamples      = 100
NumRxScanAngles = 5
NumTxScanAngles = 6

# set the TX and RX antennas to the AESA modelss
txAnt = aesaTxAnt
rxAnt = aesaRxAnt

# calculate the RX/TX antenna patterns, the angles for these patterns
# the peaks of the RX patterns, the dictionary with the peak values
# the peak values themselves and the angles at which they occur
RxAntPatterns, TxAntPatterns, \
rxtxAnglesRad, rxPeaks, \
RxAntPatMaxAngles, RxAntPatMaxValues \
    = NESZ.CalNeszAntennaPatterns_alt(
        rxAnt, 
        txAnt, 
        Wavelength, 
        NumRxScanAngles, 
        NumTxScanAngles,
        LookAngleRangeRad.min(), 
        LookAngleRangeRad.max(),
        NumSamples)

# -----------------------------------------------
# Plot the antenna patterns
if DEBUG_PRINT_PLOTS:
    fig, ax1 = plt.subplots(figsize = (8,3))
    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            # if first iteration, add the legend labels
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                     10*np.log10(TxAntPatterns[i]), color="blue", label=r"TX Elevation Pattern")
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                     10*np.log10(RxAntPatterns[i]), color="red", linestyle="dashdot", label=r"RX Elevation Pattern")
        else:
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                     10*np.log10(TxAntPatterns[i]), color="blue")
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                     10*np.log10(RxAntPatterns[i]), color="red", linestyle="dashdot")

    ax1.plot(Utils.rad2deg(RxAntPatMaxAngles/1000  + LookAngleRangeRad.min()), 
             10*np.log10(RxAntPatMaxValues), color="green", label=r"Effective RX Elevation Pattern")

    ax1.set_title("3dB Antenna pattern vs Look Angle")
    ax1.set_xlabel(r"Look Angle (deg)")
    ax1.set_ylabel(r"Antenna pattern (dB)")
    ax1.grid(linestyle='dotted')
    ax1.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# calculate the incidence angles and slant ranges over the antenna beam pattern
IA_Arr, SR_Arr = NESZ.CalcAnglesAndSlantRanges(
                    NumRxScanAngles * NumTxScanAngles, 
                    Altitude, 
                    rxtxAnglesRad, 
                    LookAngleRangeRad.min())

In [ ]:
# calculate the core angles subtended by the slant ranges, and from these the ground ranges
GR_Arr = NESZ.CalcGroundRange(
            NumRxScanAngles * NumTxScanAngles, 
            Altitude, 
            SR_Arr)

In [ ]:
# Calulate the value of NESZ of the look angles
NESZ_Arr, Nesz_max_incidence_angles, Nesz_max_values \
    = NESZ.CalcNesz(
            NumRxScanAngles * NumTxScanAngles, 
            Altitude, 
            ChirpBandwidth, 
            IA_Arr, SR_Arr, 
            TxAntPatterns, RxAntPatterns, 
            Wavelength, TxPowerPeak, 
            PulseLen, Prf)
    
Nesz_dB_Arr = 10*np.log10(NESZ_Arr)

In [ ]:
# plot the NESZ
fig, ax1 = plt.subplots(figsize = (8, 4))
# secondary axis for ground range values
ax2 = ax1.twiny()
# Add some extra space for the second axis at the bottom
fig.subplots_adjust(bottom=0.2)

# locations on the first axis, where the second axis values are calculated
new_tick_locations = np.array([15, 20., 25, 30, 35, 40, 45, 50])

# convert between the one axis to the next
def tick_function(altitude, x):
    V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
    return ["%.0f" % z for z in V / 1000]

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.2))
# sett the ticks values and locations on the second axis
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
ax2.set_xlabel("Ground Range (km)")

for i in range(NumRxScanAngles * NumTxScanAngles):
    if i == 0:
        ax1.plot(Utils.rad2deg(IA_Arr[i]), Nesz_dB_Arr[i], color="red", ls="--", label="Calculated NESZ")
    else:
        ax1.plot(Utils.rad2deg(IA_Arr[i]), Nesz_dB_Arr[i], color="red", ls="--")
ax1.plot(Utils.rad2deg(Nesz_max_incidence_angles/1000), 
         10*np.log10(Nesz_max_values), color="green", label=r"Effective NESZ")

# plot required NESZ value +- 2dB
ax1.axhline(ReqNesz, color="blue", label="Required NESZ")
ax1.fill_between(ax1.get_xlim(), ReqNesz-2, ReqNesz+2, color="yellow", alpha=0.3)

# plot the desired incidence angles limits (based on PRF analysis)
iaMax = Utils.rad2deg(SEM.IncidenceAngle(Altitude, highLa))
iaMin = Utils.rad2deg(SEM.IncidenceAngle(Altitude, lowLa))
print(np.rad2deg(lowLa))
print(iaMin, iaMax)
ax1.axvline(x=iaMin, linestyle="dashed", color="blue")
ax1.axvline(x=iaMax, linestyle="dashed", color="blue")

# ax1.set_title("NESZ vs Incidence Angle")
ax1.set_xlabel(r"Incidence Angle (deg)")
ax1.set_ylabel(r"NESZ (dB)")
ax1.grid(linestyle='dotted')
ax1.legend(framealpha=1., loc=2)
ax1.set_ylim(-32, -10)
ax1.set_xlim(15, 50)
ax1.margins(0)
ax2.set_xlim(ax1.get_xlim())

# plt.show()
if SAVE_FIGURES:
    if SAVE_FIGURE_FORMAT == "SVG":
        plt.savefig('../figures/fig_21_com_aesa_nesz.svg', format='svg', bbox_inches='tight')  
    elif SAVE_FIGURE_FORMAT == "PDF":
        plt.savefig('../figures/fig_21_com_aesa_nesz.pdf', format='pdf', bbox_inches='tight')  
    else:
        plt.savefig('../figures/fig_21_com_aesa_nesz.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
else:
    plt.show()

### AASR

In [ ]:
print("Platform velocity: {:.3f} m/s".format(SEM.PlatformVelocity(Altitude)))

In [ ]:
# Calculate the antenna patterns for AASR calculationns
NumAmbiguities = 10
FrequencySteps = 100

# Get the processed Doppler bandwidth
DopplerBandwidth = SEM.ProcessedDopplerBandwidth(SEM.PlatformVelocity(Altitude), ReqResolutionAz)
print("Processed Doppler Bandwidth: {:.2f} Hz".format(DopplerBandwidth))

# DopplerFrequencies = np.linspace(-DopplerBandwidth/2, DopplerBandwidth/2, FrequencySteps)
DopplerFrequencies = np.linspace(start=-10000, stop=10000, num=1000)

AzimuthAngles = SEM.AzimuthAngleFromDoppler(DopplerFrequencies, Wavelength, SEM.PlatformVelocity(Altitude))
                        
# calculate the TX/RX antenna patterns over the Doppler frequency bandwidth
txAzPat = txAnt.azimuth_field_pattern(Wavelength, AzimuthAngles, 0, 1, True)
rxAzPat = rxAnt.azimuth_field_pattern(Wavelength, AzimuthAngles, 0, 1, True)

# calculate the antenna power patterns
txAzPowerPat = txAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0, 1, True)
rxAzPowerPat = rxAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0, 1, True)

# calculate the scacling between the RX and TX patterns
gainScale = txAnt.gain(Wavelength) / rxAnt.gain(Wavelength)

In [ ]:
# Plot the antenna patterns
fig, ax1 = plt.subplots(figsize = (8,3))
# TX beams
ax1.plot(Utils.rad2deg(AzimuthAngles), 10*np.log10(txAzPowerPat * gainScale), color="red", label=r"TX")
# RX beam
ax1.plot(Utils.rad2deg(AzimuthAngles), 10*np.log10(rxAzPowerPat), color="black", label=r"RX", linestyle="dashed")

ax1.set_title("Noramlised 3dB Antenna azimuth power pattern vs Angle")
ax1.set_xlabel(r"Angle (deg)")
ax1.set_ylabel(r"Antenna pattern (dB)")
ax1.grid(linestyle='dotted')
ax1.set_ylim(-40, 5)
ax1.set_xlim(-1, 1)
ax1.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the AASR values
# near beam
Aasr_Arr = AASR.Get_AASR(
                DopplerBandwidth = DopplerBandwidth,
                Frequencies      = DopplerFrequencies,
                NumAmbiguities   = NumAmbiguities,
                FreqSteps        = FrequencySteps,
                Prf              = prf,
                TxAntennaPattern = txAzPat,
                RxAntennaPattern = rxAzPat,
                TxGain           = txAnt.gain(Wavelength),
                RxGain           = rxAnt.gain(Wavelength) )

print("AASR: {:.3f} dB".format(10*np.log10(Aasr_Arr)))

# create dummy ground swath array over which to plot the AASR data
SwathArr = np.linspace(np.asarray(IA_Arr).min(), np.asarray(IA_Arr).max(), 50)

# create an array equal to the ground swath array and fill with the AASR data
Aasr_Arr_Full = np.full((SwathArr.shape[0]), 10*np.log10(Aasr_Arr))

In [ ]:
# plot the AASR vs ground swath
fig, ax1 = plt.subplots(1,1, figsize = (8,4)) 

# secondary axis for ground range values
ax2 = ax1.twiny()
# Add some extra space for the second axis at the bottom
fig.subplots_adjust(bottom=0.2)

# locations on the first axis, where the second axis values are calculated
new_tick_locations = np.array([15, 20., 25., 30., 35., 40., 45., 50.])

# convert between the one axis to the next
def tick_function(altitude, x):
    V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
    return ["%.0f" % z for z in V / 1000]

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.2))
# sett the ticks values and locations on the second axis
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
ax2.set_xlabel("Ground Range (km)")

# plot the AASR values, the required AASR and the incidence angle limits
ax1.plot(np.rad2deg(SwathArr), Aasr_Arr_Full, color='red', 
         label=r"AASR with $B_\gamma$ = {:.2f} Hz".format(DopplerBandwidth))  
ax1.axhline(y=ReqAasr, label="Required AASR", color="blue")
ax1.axvline(x=iaMin, linestyle="dashed", color="blue")
ax1.axvline(x=iaMax, linestyle="dashed", color="blue")

# ax1.set_title('Azimuth-Ambiguity-to-Signal Ratio vs Slant Range')
ax1.set_ylabel(r'AASR (dB)')
ax1.set_xlabel(r'Incidence Angle (deg)')    
ax1.grid(linestyle='dotted')
ax1.set_ylim(-22, -1)
ax1.set_xlim(15, 50)
ax1.legend(framealpha=1.)
ax1.margins(0)

ax2.set_xlim(ax1.get_xlim())

if SAVE_FIGURES:
    if SAVE_FIGURE_FORMAT == "SVG":
        plt.savefig('../figures/fig_22_com_aesa_aasr.svg', format='svg', bbox_inches='tight')  
    elif SAVE_FIGURE_FORMAT == "PDF":
        plt.savefig('../figures/fig_22_com_aesa_aasr.pdf', format='pdf', bbox_inches='tight')  
    else:
        plt.savefig('../figures/fig_22_com_aesa_aasr.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
else:
    plt.show()

### RASR

In [ ]:
# calcuate the RASSR values for the planar array
if is_main_module():
    # sest the number of samples and scan angles in TX and RX
    NumSamples = 100
    NumRxScanAngles = 5
    NumTxScanAngles = 6
    # set the near range and far range number of ambiguities
    NF = 5
    NN = 3

    # set the RX/TX antennas to the AESA model
    txAnt = aesaTxAnt
    rxAnt = aesaRxAnt

    #------------------------------------------------------------
    # firstly get the antenna patterns and anges for the main antenna beams
    RxAntPatterns, TxAntPatterns, \
    rxtxAnglesRad, rxPeaks, \
    RxAntPatMaxAngles, RxAntPatMaxValues \
        = RASR.CalRasrMainAntennaPatterns_alt(
            rxAnt, 
            txAnt, 
            Wavelength, 
            NumRxScanAngles, 
            NumTxScanAngles,
            LookAngleRangeRad.min(), 
            LookAngleRangeRad.max(), 
            NumSamples)
    
    # Plot the main antenna patterns
    if DEBUG_PRINT_PLOTS:
        fig, ax1 = plt.subplots(figsize = (8,3))
        for i in range(NumRxScanAngles * NumTxScanAngles):
            if i == 0:
                # when it is the first iteration, add the legend labels
                ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                         10*np.log10(TxAntPatterns[i]), color="red", label="TX Elevation")
                ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                         10*np.log10(RxAntPatterns[i]), color="black", linestyle="dashed", label="RX Elevation")
            else:
                ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                         10*np.log10(TxAntPatterns[i]), color="red")
                ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                         10*np.log10(RxAntPatterns[i]), color="black", linestyle="dashed")

        ax1.set_title("3dB Antenna pattern vs Angle")
        ax1.set_xlabel(r"Angle (deg)")
        ax1.set_ylabel(r"Antenna pattern (dB)")
        ax1.grid(linestyle='dotted')
        plt.tight_layout()
        ax1.legend()
        plt.show()

    #------------------------------------------------------------
    # calculate the incidence angles and slant ranges over the antenna beam pattern
    IA_Arr, SR_Arr = NESZ.CalcAnglesAndSlantRanges(
        NumRxScanAngles * NumTxScanAngles, 
        Altitude, 
        rxtxAnglesRad, 
        LookAngleRangeRad.min())

    #------------------------------------------------------------
    # calculate the main RASR values for each slant range
    RasrMain = RASR.Get_RasrMain(
        SlantRange = SR_Arr,
        IncidenceAngle = IA_Arr,
        TxAntennaPattern = np.asarray(TxAntPatterns), 
        RxAntennaPattern = np.asarray(RxAntPatterns) )
    RasrMain_dB = 10*np.log10(RasrMain)
    
    # Plot RASR Main values
    if DEBUG_PRINT_PLOTS:
        fig, ax1 = plt.subplots(figsize = (8, 3))
        for i in range(NumRxScanAngles * NumTxScanAngles):
            if i == 0:
                ax1.plot(Utils.rad2deg(IA_Arr[i]), RasrMain_dB[i], color="red", ls="--", label="RASR")
            else:
                ax1.plot(Utils.rad2deg(IA_Arr[i]), RasrMain_dB[i], color="red", ls="--")

        ax1.axhline(ReqNesz, color="blue", label="Required RASR")
        ax1.set_title("RASR vs Slant Range")
        ax1.set_xlabel(r"Incidence Angle (deg)")
        ax1.set_ylabel(r"RASR (dB)")
        ax1.grid(linestyle='dotted')
        ax1.legend()
        plt.tight_layout()
        plt.show()

    #------------------------------------------------------------
    # Calculate the slant ranges, incidence angles, and RX/TX antenna patterns over the ambiguous PRFs
    TX_Amb_Arr, RX_Amb_Arr, SR_Amb_Arr, IA_Amb_Arr \
        = RASR.CalcAmbiguousAntennaPatterns_alt(
            rxAnt, 
            txAnt, 
            NumRxScanAngles * NumTxScanAngles,
            SR_Arr, 
            NN, NF, 
            Altitude, 
            AntennaOffset, 
            Wavelength, 
            prf)

    # ----------------------------------------------------------
    # for each slant range and each ambiguity count, calculate the ambiguous value
    Rasr_Amb_Scan_Arr = []
    for i in range(NumRxScanAngles * NumTxScanAngles):
        Rasr_Amb_Arr = []
        for j in range(SR_Amb_Arr[i].shape[0]):
            Rasr_Amb_Arr.append(
                RASR.RasrAmb(
                    TX_Amb_Arr[i][j], 
                    RX_Amb_Arr[i][j], 
                    SR_Amb_Arr[i][j], 
                    IA_Amb_Arr[i][j]) )
        Rasr_Amb_Scan_Arr.append(Rasr_Amb_Arr)
    Rasr_Amb_Scan_Arr = np.asarray(Rasr_Amb_Scan_Arr)

    # ----------------------------------------------------------
    # add all the ambiguities for a given slant range together,
    # this is the total ambiguity singal for a given slant range
    Rasr_Amb_sum = []

    for i in range(NumRxScanAngles * NumTxScanAngles):
        Rasr_Amb_sum.append(np.nansum(Rasr_Amb_Scan_Arr[i], axis=0))
            
    Rasr_Amb_sum = np.asarray(Rasr_Amb_sum)
    # print(Rasr_Amb_sum.shape)

    # ----------------------------------------------------------
    # calculate the Total RASR value for each slant range
    RASR_Total_Arr = [] = []
    for i in range(NumRxScanAngles * NumTxScanAngles):
        RASR_Arr = []
        for j in range(SR_Arr[i].shape[0]):
            RASR_Arr.append(RasrMain[i][j] * Rasr_Amb_sum[i][j])
        RASR_Total_Arr.append(RASR_Arr)

    RASR_Total_Arr = np.asarray(RASR_Total_Arr)
    RASR_Arr_dB = 10*np.log10(RASR_Total_Arr)

    # ----------------------------------------------------------
    # Plot RASR values vs ground
    fig, ax1 = plt.subplots(figsize = (8, 4))

    # secondary axis for ground range values
    ax2 = ax1.twiny()
    # Add some extra space for the second axis at the bottom
    fig.subplots_adjust(bottom=0.2)

    # locations on the first axis, where the second axis values are calculated
    new_tick_locations = np.array([15, 20., 25., 30., 35., 40., 45., 50.])

    # convert between the one axis to the next
    def tick_function(altitude, x):
        V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
        return ["%.0f" % z for z in V / 1000]

    # Move twinned axis ticks and label from top to bottom
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")

    # Offset the twin axis below the host
    ax2.spines["bottom"].set_position(("axes", -0.2))
    # sett the ticks values and locations on the second axis
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
    ax2.set_xlabel("Ground Range (km)")

    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), RASR_Arr_dB[i], color="red", ls="--", label="RASR")
        else:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), RASR_Arr_dB[i], color="red", ls="--")

    ax1.axhline(ReqNesz, color="blue", label="Required RASR")
    ax1.axvline(x=iaMin, linestyle="dashed", color="blue")
    ax1.axvline(x=iaMax, linestyle="dashed", color="blue")

    # ax1.set_title("RASR vs Slant Range")
    ax1.set_xlabel(r"Incidence Angle (deg)")
    ax1.set_ylabel(r"RASR (dB)")
    ax1.grid(linestyle='dotted')
    ax1.legend(framealpha=1.)
    ax1.set_xlim(15, 50)
    ax1.margins(0)
    ax2.set_xlim(ax1.get_xlim())
    
    # plt.show()
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_23_com_aesa_rasr.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_23_com_aesa_rasr.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_23_com_aesa_rasr.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

## Reflector Design

In [ ]:
# main reflector antenna parameters
if is_main_module():
    AntennaDiameter = 4.416    # antenna diameter is equal to the planar array antenna azimuth length
    NumAzChannels   = 1        # Number of azimuth channels. Each consisting of NumElChannels in elevation
    F_on_D          = 0.6      # antenna focal point location
    F               = AntennaDiameter * F_on_D

    # set the look angle range to the observable range from the PRF analysis
    LookAngleRange = np.arange(32.5, 37.1, 0.1)
    LookAngleRangeRad = Utils.deg2rad(LookAngleRange)
    
    # calculate the incidence angle range from this look angle range
    IncidenceAngleRangeRad = SEM.IncidenceAngle(Altitude, LookAngleRangeRad)
    IncidenceAngleRange = Utils.rad2deg(IncidenceAngleRangeRad)

In [ ]:
# TX Antenna code
TX_efficiency   = 0.6  # reflector antenna efficiency
TX_BW           = aesaTxAnt.elevation_beamwidth(Wavelength) # same beamwidth as the phased array
print("TX Beamwidth: {:.2f} deg".format(Utils.rad2deg(TX_BW)))

# the antenna elevation dimension is estimated from the beamwidth required to illuminate the entire swath
TX_elv_dim = RA.RectangularAntenna.dimension_for_beamwidth(Wavelength, TX_BW, TX_efficiency)
print("TX sub-reflector height: {:.2f} m".format(TX_elv_dim))

# calculate the angles with which the feeds illuminate the reflector, and the feed-array length
feed_angle = TX_elv_dim / F
feed_efficiency = 0.7
feed_arr_len = RA.RectangularAntenna.dimension_for_beamwidth(Wavelength, feed_angle, feed_efficiency)
print("Feed angle: {:.2f} deg".format(Utils.rad2deg(feed_angle)))
print("Feed length: {:.3f} m".format(feed_arr_len))

# Calculate the number of feeds using and ideal lambda/2 spacing
num_feeds = int(feed_arr_len / (Wavelength/2))

# set the number of feeds (channels) for later use
NumElChannels = num_feeds
NumRxBeams = num_feeds

# calculate the actual feed spacing
feed_spacing = feed_arr_len / num_feeds
print("Feed spacing: {:.3f} m".format(feed_spacing))

# initialise the TX object
reflectorTxAnt = RA.RectangularAntenna(
                    length = AntennaDiameter, 
                    height = TX_elv_dim, 
                    efficiency= TX_efficiency)

TX_swath, TX_gswath =  SEM.SwathGroundSwathFromBeamwidth(Altitude, Utils.deg2rad(AntennaOffset), TX_BW)
TX_gain = reflectorTxAnt.gain(Wavelength)

In [ ]:
# display the TX antenna characteristics in a table format
df = pd.DataFrame (
    {
        "Parameter" : ['Diameter', 
                       'Height (Elevation)', 
                       'Elevation Feed Spacing', 
                       '# TX Elevation Feeds', 
                       'TX Elevation Beamwidth',
                       '# TX Azimuth Feeds',  
                       'TX Azimuth Beamwidth',
                       'TX Gain', 
                       'Ground Swath'],
        "Value" : [reflectorTxAnt.length(), 
                   reflectorTxAnt.height(),
                   feed_spacing, 
                   num_feeds,
                   Utils.rad2deg(reflectorTxAnt.elevation_beamwidth(Wavelength)),
                   NumAzChannels, 
                   Utils.rad2deg(reflectorTxAnt.azimuth_beamwidth(Wavelength)),
                   10*np.log10(reflectorTxAnt.gain(Wavelength)), 
                   TX_gswath / 1000],
        "Unit" : ['m', 'm', 
                  'm', 
                  '','deg', 
                  '','deg', 
                  'dBi', 'km']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles) \
  .style.applymap(yellow, subset=pd.IndexSlice[0:2, :]) \
        .applymap(blue, subset=pd.IndexSlice[3:6, :]) \
        .applymap(green, subset=pd.IndexSlice[7:8, :])

In [ ]:
# Look angle and incidence angle calculations for (feed) subswaths
initial_look_angle = LookAngleRange.min()
initial_beam_width_Deg = Utils.rad2deg((TX_BW) / (NumRxBeams))
print("RX single feed beamwidth: {:.3f} deg".format(initial_beam_width_Deg))

# calculate the beamwidth offset for each feed
Beam_look_angle_Deg = []
for i in range(NumRxBeams):
     Beam_look_angle_Deg.append(initial_look_angle + i*initial_beam_width_Deg)

Beam_look_angle_Deg = np.asarray(Beam_look_angle_Deg)
Beam_look_angle = Utils.deg2rad(Beam_look_angle_Deg)

# calculate the incidence angles for these look angles
Beam_incidence_angle = SEM.IncidenceAngle(Altitude, Beam_look_angle)
Beam_incidence_angle_Deg = Utils.rad2deg(Beam_incidence_angle)

# mid-swath incidence angle calculated from the antenna boresight look angle/mounting angle
antenna_normal_incidence_angle = SEM.IncidenceAngle(Altitude, Utils.deg2rad(AntennaOffset))

In [ ]:
# initialise the RX antenna object
RX_efficiency      = 0.6  # same as TX antenna
RX_BW_sub_aperture = initial_beam_width_Deg  # degrees

# the antenna diameter is estimated from the beamwidth
RX_diameter_sub_aperture = (RX_efficiency * Wavelength) / Utils.deg2rad(RX_BW_sub_aperture)

# the gain however is determined by the full surface of the reflector
Area_full = math.pi * (AntennaDiameter/2)**2
RX_gain_full = (4*math.pi*RX_efficiency * Area_full) / (math.pow(Wavelength, 2))

Reflector_RX_Beams = []
for i in range(NumRxBeams):
    Reflector_RX_Beams.append(RA.RectangularAntenna(
        length = AntennaDiameter, 
        height = RX_diameter_sub_aperture, 
        efficiency = RX_efficiency))

# swath covered by RX beam
RX_ground_swaths = []
for i in range(NumRxBeams):
    _, rxGroundSwath = SEM.SwathGroundSwathFromBeamwidth(Altitude, Beam_look_angle[i], Utils.deg2rad(RX_BW_sub_aperture))
    RX_ground_swaths.append(rxGroundSwath)

In [ ]:
# display the RX antenna characteristics in a table format
avg_swath = np.average(RX_ground_swaths, axis=0)
df = pd.DataFrame (
    {
        "Parameter" : ['Diameter', 
                       '# RX Elevation Feeds', 
                       'RX Feed Elevation Beamwidth',
                       '# RX Azimuth Feeds',  
                       'RX Azimuth Beamwidth',
                       'RX Gain', 
                       'Average Feed Ground Swath'],
        "Value" : [AntennaDiameter, 
                   NumRxBeams,
                   Utils.rad2deg(Reflector_RX_Beams[0].elevation_beamwidth(Wavelength)),
                   NumAzChannels, 
                   Utils.rad2deg(Reflector_RX_Beams[0].azimuth_beamwidth(Wavelength)),
                   10*np.log10(RX_gain_full), 
                   RX_ground_swaths[0] / 1000],
        "Unit" : ['m',
                  '','deg', 
                  '','deg', 
                  'dBi', 'km']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles) \
  .style.applymap(yellow, subset=pd.IndexSlice[0:0, :]) \
        .applymap(blue, subset=pd.IndexSlice[1:4, :]) \
        .applymap(green, subset=pd.IndexSlice[5:6, :])

In [ ]:
# antenna pattern for TX and RX antennas
AntennaPatternRange = np.arange(-90, 90, 0.1)

# Azimuth angles used only to visualise the antenna pattern in azimuth
azimuthAngles = np.arange(-5, 5, 0.01)   # azimuth angles in degrees

txAnt = reflectorTxAnt

# ---------------------------------------------------
# TX antenna power pattern
txElvPat = txAnt.elevation_power_pattern(Wavelength, Utils.deg2rad(AntennaPatternRange), 0)
txAzPat  = txAnt.azimuth_power_pattern(Wavelength, Utils.deg2rad(azimuthAngles), 0)

# RX antenna power patterns
rxElvPat = []
rxAzPat = []
for i in range(NumRxBeams):
    rxElvPat.append(Reflector_RX_Beams[i].elevation_power_pattern(Wavelength, Utils.deg2rad(AntennaPatternRange), 0))
    rxAzPat.append(Reflector_RX_Beams[i].azimuth_power_pattern(Wavelength, Utils.deg2rad(azimuthAngles), 0))

# TX / RX antenna gain scaling factor
gainScales = []
for i in range(NumRxBeams):
    # use the entire reflector surface for the RX gain
    gainScales.append(txAnt.gain(Wavelength) / RX_gain_full)

# ---------------------------------------------------
# Plot the antenna patterns
fig, (ax1, ax2) = plt.subplots(2,1, figsize = (8,6))
# Elevation pattern
# TX pattern
ax1.plot(AntennaPatternRange, 10*np.log10(txElvPat * gainScales[0]), color="blue", label=r"TX")                  
# rx patterns
for i in range(NumRxBeams):
    RX_BW = Utils.rad2deg(Reflector_RX_Beams[i].elevation_beamwidth(Wavelength))
    if i == 0:
        # when its the first iteration, add the label
        ax1.plot(AntennaPatternRange + (-((NumRxBeams-1)/2)*RX_BW) + (i*RX_BW), 
                 10*np.log10(rxElvPat[i]), linestyle="dashed", label=r"RX")
    else:
        ax1.plot(AntennaPatternRange + (-((NumRxBeams-1)/2)*RX_BW) + (i*RX_BW), 
                 10*np.log10(rxElvPat[i]), linestyle="dashed")

# labels, legends and grids
ax1.legend(loc=2)
ax1.set_title(r"Normalised Planar Array Elevation Antenna Pattern, $G_{el}(\theta)$ vs Angle")
ax1.set_xlabel(r"Angle from Antenna Center (deg)")
ax1.set_ylabel(r"Antenna pattern, $G_{el}(\theta)$ (dB)")
ax1.grid(linestyle='dotted')

# set the figure area limits
ax1.set_ylim(-40, 5)
ax1.set_xlim(-15, 15)

# azimuth patterns
# TX pattern
ax2.plot(azimuthAngles, 10*np.log10(txAzPat * gainScales[0]), color="blue", label=r"TX")
# RX patterns
for i in range(NumRxBeams):
    if i == 0:
        ax2.plot(azimuthAngles, 10*np.log10(rxAzPat[i]), linestyle="dashed", color="green", label=r"RX")
    else:
        ax2.plot(azimuthAngles, 10*np.log10(rxAzPat[i]), linestyle="dashed", color="green")

ax2.legend(loc=2)
ax2.set_title(r"Normalised Planar Array Azimuth Power Pattern, $G_{az}(\theta)$ vs Angle")
ax2.set_xlabel(r"Angle from Antenna Center (deg)")
ax2.set_ylabel(r"Antenna pattern, $G_{az}(\theta)$ (dB)")
ax2.grid(linestyle='dotted')
ax2.set_ylim(-40, 5)
ax2.set_xlim(azimuthAngles.min(), azimuthAngles.max())

plt.tight_layout()
plt.show()

### NESZ

In [ ]:
# reset the incidence angle ranges
IncidenceAngleRange = np.arange(20, 49.1, 0.1) # Scene incidence angle range in degrees
IncidenceAngleRangeRad = IncidenceAngleRange * (np.pi/180)

LookAngleRangeRad = SEM.LookAngle(Altitude, IncidenceAngleRangeRad)
LookAngleRange = LookAngleRangeRad * (180/np.pi)

In [ ]:
# instantiate the RX antenna object
RX_efficiency      = 0.6
reflectorRxAnt = RA.RectangularAntenna(
                    length = AntennaDiameter, 
                    height = RX_diameter_sub_aperture, 
                    efficiency= RX_efficiency)

RX_swath, RX_gswath =  SEM.SwathGroundSwathFromBeamwidth(Altitude, Utils.deg2rad(AntennaOffset), RX_BW)
# The full surface of the reflector is used for gain on receive
RX_gain = RX_gain_full

In [ ]:
# Calculate the antenna patterns used for calculating the NESZ
# set the number of samples, the number of TX and RX scan angles
NumSamples = 100
NumRxScanAngles = 9
NumTxScanAngles = 5

# set the TX/RX antenna to the reflector antenna models
txAnt = reflectorTxAnt
rxAnt = reflectorRxAnt

# calculate the antenna patterns and angles used for the NESZ calculations
RxAntPatterns, TxAntPatterns, \
rxtxAnglesRad, rxPeaks, \
RxAntPatMaxAngles, RxAntPatMaxValues \
    = NESZ.CalNeszAntennaPatterns_rect(
        rxAnt, 
        txAnt, 
        Wavelength, 
        NumRxScanAngles, 
        NumTxScanAngles,
        LookAngleRangeRad.min(), 
        LookAngleRangeRad.max(),
        NumSamples,
        TX_gain,
        RX_gain_full)

# -----------------------------------------------
# Plot the antenna patterns
if DEBUG_PRINT_PLOTS:
    fig, ax1 = plt.subplots(figsize = (8,3))
    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            # when its the first iteration, add the legend labels
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                     10*np.log10(TxAntPatterns[i]), color="blue", label=r"TX Elevation Pattern")
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                     10*np.log10(RxAntPatterns[i]), color="red", linestyle="dashdot", label=r"RX Elevation Pattern")
        else:
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                     10*np.log10(TxAntPatterns[i]), color="blue")
            ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                     10*np.log10(RxAntPatterns[i]), color="red", linestyle="dashdot")

    ax1.set_title("3dB Antenna pattern vs Look Angle")
    ax1.set_xlabel(r"Look Angle (deg)")
    ax1.set_ylabel(r"Antenna pattern (dB)")
    ax1.grid(linestyle='dotted')
    ax1.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# calculate the incidence angles and slant ranges over the antenna beam pattern
IA_Arr, SR_Arr = NESZ.CalcAnglesAndSlantRanges(
                    NumRxScanAngles * NumTxScanAngles, 
                    Altitude, 
                    rxtxAnglesRad, 
                    LookAngleRangeRad.min())

In [ ]:
# calculate the core angles subtended by the slant ranges, and from these the ground ranges
GR_Arr = NESZ.CalcGroundRange(
            NumRxScanAngles * NumTxScanAngles, 
            Altitude, 
            SR_Arr)

In [ ]:
# Calulate the value of NESZ of the look angles
NESZ_Arr, Nesz_max_incidence_angles, Nesz_max_values \
    = NESZ.CalcNesz(
            NumRxScanAngles * NumTxScanAngles, 
            Altitude, 
            ChirpBandwidth, 
            IA_Arr, SR_Arr, 
            TxAntPatterns, RxAntPatterns, 
            Wavelength, TxPowerPeak, 
            PulseLen, Prf)
    
Nesz_dB_Arr = 10*np.log10(NESZ_Arr)

In [ ]:
# plot the NESZ
fig, ax1 = plt.subplots(figsize = (8, 4))
# secondary axis for ground range values
ax2 = ax1.twiny()
# Add some extra space for the second axis at the bottom
fig.subplots_adjust(bottom=0.2)

# locations on the first axis, where the second axis values are calculated
new_tick_locations = np.array([15, 20., 25, 30, 35, 40, 45, 50])

# convert between the one axis to the next
def tick_function(altitude, x):
    V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
    return ["%.0f" % z for z in V / 1000]

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.2))
# sett the ticks values and locations on the second axis
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
ax2.set_xlabel("Ground Range (km)")

for i in range(NumRxScanAngles * NumTxScanAngles):
    if i == 0:
        ax1.plot(Utils.rad2deg(IA_Arr[i]), Nesz_dB_Arr[i], color="red", ls="--", label="Calculated NESZ")
    else:
        ax1.plot(Utils.rad2deg(IA_Arr[i]), Nesz_dB_Arr[i], color="red", ls="--")
ax1.plot(Utils.rad2deg(Nesz_max_incidence_angles/1000), 
         10*np.log10(Nesz_max_values), color="green", label=r"Effective NESZ")

# plot required NESZ value +- 2dB
ax1.axhline(ReqNesz, color="blue", label="Required NESZ")
ax1.fill_between((15, 50), ReqNesz-2, ReqNesz+2, color="yellow", alpha=0.3)

# plot the desired incidence angles limits (based on PRF analysis)
iaMax = Utils.rad2deg(SEM.IncidenceAngle(Altitude, highLa))
iaMin = Utils.rad2deg(SEM.IncidenceAngle(Altitude, lowLa))
print(np.rad2deg(lowLa))
print(iaMin, iaMax)
ax1.axvline(x=iaMin, linestyle="dashed", color="blue")
ax1.axvline(x=iaMax, linestyle="dashed", color="blue")

# ax1.set_title("NESZ vs Incidence Angle")
ax1.set_xlabel(r"Incidence Angle (deg)")
ax1.set_ylabel(r"NESZ (dB)")
ax1.grid(linestyle='dotted')
ax1.legend(framealpha=1.)
ax1.set_ylim(-33, -18)
ax1.set_xlim(15, 50)
ax1.margins(0)
ax2.set_xlim(ax1.get_xlim())

# plt.show()
if SAVE_FIGURES:
    if SAVE_FIGURE_FORMAT == "SVG":
        plt.savefig('../figures/fig_24_com_refl_nesz.svg', format='svg', bbox_inches='tight')  
    elif SAVE_FIGURE_FORMAT == "PDF":
        plt.savefig('../figures/fig_24_com_refl_nesz.pdf', format='pdf', bbox_inches='tight')  
    else:
        plt.savefig('../figures/fig_24_com_refl_nesz.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
else:
    plt.show()

### AASR

In [ ]:
# Calculate the antenna patterns for AASR calculationns
NumAmbiguities = 10
FrequencySteps = 100

# Get the processed Doppler bandwidth
DopplerBandwidth = SEM.ProcessedDopplerBandwidth(SEM.PlatformVelocity(Altitude), ReqResolutionAz)
print("Processed Doppler Bandwidth: {:.2f} Hz".format(DopplerBandwidth))

# DopplerFrequencies = np.linspace(-DopplerBandwidth/2, DopplerBandwidth/2, FrequencySteps)
DopplerFrequencies = np.linspace(start=-10000, stop=10000, num=1000)

AzimuthAngles = SEM.AzimuthAngleFromDoppler(DopplerFrequencies, Wavelength, SEM.PlatformVelocity(Altitude))
                        
# calculate the TX/RX antenna patterns over the Doppler frequency bandwidth
txAzPat = txAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0)
rxAzPat = rxAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0)
# calculate the scaling between the RX and TX antennas
gainScale = txAnt.gain(Wavelength) / rxAnt.gain(Wavelength)

In [ ]:
# Plot the antenna patterns
fig, ax1 = plt.subplots(figsize = (8,3))
# TX beams
ax1.plot(Utils.rad2deg(AzimuthAngles), 10*np.log10(txAzPat * gainScale), color="red", label=r"TX")
# RX beam
ax1.plot(Utils.rad2deg(AzimuthAngles), 10*np.log10(rxAzPat), color="black", label=r"RX", linestyle="dashed")

ax1.set_title("Noramlised 3dB Antenna azimuth power pattern vs Angle")
ax1.set_xlabel(r"Angle (deg)")
ax1.set_ylabel(r"Antenna pattern (dB)")
ax1.grid(linestyle='dotted')
ax1.set_ylim(-40, 5)
ax1.set_xlim(-1, 1)
ax1.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the AASR values
Aasr_Arr = AASR.Get_AASR(
                DopplerBandwidth = DopplerBandwidth,
                Frequencies      = DopplerFrequencies,
                NumAmbiguities   = NumAmbiguities,
                FreqSteps        = FrequencySteps,
                Prf              = prf,
                TxAntennaPattern = txAzPat,
                RxAntennaPattern = rxAzPat,
                TxGain           = txAnt.gain(Wavelength),
                RxGain           = RX_gain_full)

print("AASR: {:.3f} dB".format(10*np.log10(Aasr_Arr)))

# create dummy ground swath array over which to plot the AASR data
SwathArr = np.linspace(np.asarray(IA_Arr).min(), np.asarray(IA_Arr).max(), 50)

# create an array equal to the ground swath array and fill with the AASR data
Aasr_Arr_Full = np.full((SwathArr.shape[0]), 10*np.log10(Aasr_Arr))

In [ ]:
# plot the AASR vs ground swath
fig, ax1 = plt.subplots(1,1, figsize = (8,4))    
ax1.plot(np.rad2deg(SwathArr), Aasr_Arr_Full, color='red', label=r"AASR with $B_\gamma$ = {:.2f} Hz"
         .format(DopplerBandwidth))  
ax1.axhline(y=ReqAasr, label="Required AASR", color="blue")
ax1.axvline(x=iaMin, linestyle="dashed", color="blue")
ax1.axvline(x=iaMax, linestyle="dashed", color="blue")

# ax1.set_title('Azimuth-Ambiguity-to-Signal Ratio vs Slant Range')
ax1.set_ylabel(r'AASR (dB)')
ax1.set_xlabel(r'Incidence Angle (deg)')    
plt.grid(linestyle='dotted')
ax1.set_ylim(-22, -6)
ax1.set_xlim(15, 50)
ax1.legend(framealpha=1.)
ax1.margins(0)

# plt.show()
if SAVE_FIGURES:
    if SAVE_FIGURE_FORMAT == "SVG":
        plt.savefig('../figures/fig_25_com_refl_aasr.svg', format='svg', bbox_inches='tight')  
    elif SAVE_FIGURE_FORMAT == "PDF":
        plt.savefig('../figures/fig_25_com_refl_aasr.pdf', format='pdf', bbox_inches='tight')  
    else:
        plt.savefig('../figures/fig_25_com_refl_aasr.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
else:
    plt.show()

### RASR

In [ ]:
# calculate the RASR values for the reflector antenna
if is_main_module():
    # set the number of sampes, TX/RX scan angles and the number of ambiguities in the near and far ranges
    NumSamples = 100
    NumRxScanAngles = 9
    NumTxScanAngles = 6
    NF = 5
    NN = 3

    # set the TX/RX antenna to the reflector models
    txAnt = reflectorTxAnt
    rxAnt = reflectorRxAnt

    #------------------------------------------------------------
    # firstly get the antenna patterns and anges for the main antenna beams
    RxAntPatterns, TxAntPatterns, \
    rxtxAnglesRad, rxPeaks, \
    RxAntPatMaxAngles, RxAntPatMaxValues \
        = NESZ.CalNeszAntennaPatterns_rect(
            rxAnt, 
            txAnt, 
            Wavelength, 
            NumRxScanAngles, 
            NumTxScanAngles,
            LookAngleRangeRad.min(), 
            LookAngleRangeRad.max(),
            NumSamples,
            TX_gain,
            RX_gain_full)
    
    # Plot the main antenna patterns
    if DEBUG_PRINT_PLOTS:
        fig, ax1 = plt.subplots(figsize = (8,3))
        for i in range(NumRxScanAngles * NumTxScanAngles):
            if i == 0:
                # when its the first iteration, add the legend labels
                ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                         10*np.log10(TxAntPatterns[i]), color="red", label="TX Elevation")
                ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                         10*np.log10(RxAntPatterns[i]), color="black", linestyle="dashed", label="RX Elevation")
            else:
                ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                         10*np.log10(TxAntPatterns[i]), color="red")
                ax1.plot(Utils.rad2deg(rxtxAnglesRad[i] + LookAngleRangeRad.min()), 
                         10*np.log10(RxAntPatterns[i]), color="black", linestyle="dashed")

        ax1.set_title("3dB Antenna pattern vs Angle")
        ax1.set_xlabel(r"Angle (deg)")
        ax1.set_ylabel(r"Antenna pattern (dB)")
        ax1.grid(linestyle='dotted')
        plt.tight_layout()
        ax1.legend()
        plt.show()

    #------------------------------------------------------------
    # calculate the incidence angles and slant ranges over the antenna beam pattern
    IA_Arr, SR_Arr = NESZ.CalcAnglesAndSlantRanges(
        NumRxScanAngles * NumTxScanAngles, 
        Altitude, 
        rxtxAnglesRad, 
        LookAngleRangeRad.min())

    #------------------------------------------------------------
    # calculate the main RASR values for each slant range
    RasrMain = RASR.Get_RasrMain(
        SlantRange = SR_Arr,
        IncidenceAngle = IA_Arr,
        TxAntennaPattern = np.asarray(TxAntPatterns), 
        RxAntennaPattern = np.asarray(RxAntPatterns) )
    RasrMain_dB = 10*np.log10(RasrMain)
    
    # Plot RASR Main values
    if DEBUG_PRINT_PLOTS:
        fig, ax1 = plt.subplots(figsize = (8, 3))
        for i in range(NumRxScanAngles * NumTxScanAngles):
            if i == 0:
                ax1.plot(Utils.rad2deg(IA_Arr[i]), RasrMain_dB[i], color="red", ls="--", label="RASR")
            else:
                ax1.plot(Utils.rad2deg(IA_Arr[i]), RasrMain_dB[i], color="red", ls="--")

        ax1.axhline(ReqNesz, color="blue", label="Required RASR")
        ax1.set_title("RASR vs Slant Range")
        ax1.set_xlabel(r"Incidence Angle (deg)")
        ax1.set_ylabel(r"RASR (dB)")
        ax1.grid(linestyle='dotted')
        ax1.legend()
        plt.tight_layout()
        plt.show()

    #------------------------------------------------------------
    # Calculate the slant ranges, incidence angles, and RX/TX antenna patterns over the ambiguous PRFs
    TX_Amb_Arr, RX_Amb_Arr, SR_Amb_Arr, IA_Amb_Arr \
        = RASR.CalcAmbiguousAntennaPatterns_rect(
            rxAnt, 
            txAnt, 
            NumRxScanAngles * NumTxScanAngles,
            SR_Arr, 
            NN, NF, 
            Altitude, 
            AntennaOffset, 
            Wavelength, 
            prf,
            TX_gain,
            RX_gain_full)

    # ----------------------------------------------------------
    # for each slant range and each ambiguity count, calculate the ambiguous value
    Rasr_Amb_Scan_Arr = []
    for i in range(NumRxScanAngles * NumTxScanAngles):
        Rasr_Amb_Arr = []
        for j in range(SR_Amb_Arr[i].shape[0]):
            Rasr_Amb_Arr.append(
                RASR.RasrAmb(
                    TX_Amb_Arr[i][j], 
                    RX_Amb_Arr[i][j], 
                    SR_Amb_Arr[i][j], 
                    IA_Amb_Arr[i][j]) )
        Rasr_Amb_Scan_Arr.append(Rasr_Amb_Arr)
    Rasr_Amb_Scan_Arr = np.asarray(Rasr_Amb_Scan_Arr)

    # ----------------------------------------------------------
    # add all the ambiguities for a given slant range together,
    # this is the total ambiguity singal for a given slant range
    Rasr_Amb_sum = []

    for i in range(NumRxScanAngles * NumTxScanAngles):
        Rasr_Amb_sum.append(np.nansum(Rasr_Amb_Scan_Arr[i], axis=0))
            
    Rasr_Amb_sum = np.asarray(Rasr_Amb_sum)
    # print(Rasr_Amb_sum.shape)

    # ----------------------------------------------------------
    # calculate the Total RASR value for each slant range
    RASR_Total_Arr = [] = []
    for i in range(NumRxScanAngles * NumTxScanAngles):
        RASR_Arr = []
        for j in range(SR_Arr[i].shape[0]):
            RASR_Arr.append(RasrMain[i][j] * Rasr_Amb_sum[i][j])
        RASR_Total_Arr.append(RASR_Arr)

    RASR_Total_Arr = np.asarray(RASR_Total_Arr)
    RASR_Arr_dB = 10*np.log10(RASR_Total_Arr)

    # ----------------------------------------------------------
    # Plot RASR values vs ground
    fig, ax1 = plt.subplots(figsize = (8, 4))

    # secondary axis for ground range values
    ax2 = ax1.twiny()
    # Add some extra space for the second axis at the bottom
    fig.subplots_adjust(bottom=0.2)

    # locations on the first axis, where the second axis values are calculated
    new_tick_locations = np.array([15, 20., 25., 30., 35., 40., 45., 50.])

    # convert between the one axis to the next
    def tick_function(altitude, x):
        V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
        return ["%.0f" % z for z in V / 1000]

    # Move twinned axis ticks and label from top to bottom
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")

    # Offset the twin axis below the host
    ax2.spines["bottom"].set_position(("axes", -0.2))
    # sett the ticks values and locations on the second axis
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
    ax2.set_xlabel("Ground Range (km)")

    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), RASR_Arr_dB[i], color="red", ls="--", label="RASR")
        else:
            ax1.plot(Utils.rad2deg(IA_Arr[i]), RASR_Arr_dB[i], color="red", ls="--")

    ax1.axhline(ReqNesz, color="blue", label="Required RASR")
    ax1.axvline(x=iaMin, linestyle="dashed", color="blue")
    ax1.axvline(x=iaMax, linestyle="dashed", color="blue")

    # ax1.set_title("RASR vs Slant Range")
    ax1.set_xlabel(r"Incidence Angle (deg)")
    ax1.set_ylabel(r"RASR (dB)")
    ax1.grid(linestyle='dotted')
    ax1.legend(framealpha=1.)
    ax1.set_xlim(15, 50)
    ax1.margins(0)
    ax2.set_xlim(ax1.get_xlim())
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_26_com_refl_rasr.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_26_com_refl_rasr.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_26_com_refl_rasr.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()